In [1]:
# functions here are copied from feature_extraction.ipynb because it got too messy
# this notebook only serves as a fresh slate for work previously done

In [3]:
# imports

from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical, plot_model
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
import os
from pickle import dump, load
import numpy as np
import pandas as pd
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import argparse

from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

2022-04-18 18:09:46.929505: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-18 18:09:46.929524: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/tmp/ipykernel_7416/2631900983.py:24: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  tqdm().pandas()


0it [00:00, ?it/s]

In [4]:
pwd

'/home/mattiboi/Desktop/Data Science/Metis/Deep Learning/project'

In [28]:
training_features = load(open('data/features.p', 'rb'))
validation_features = load(open('data/val_features.p', 'rb'))
img_names_text = 'img_names.txt'
val_img_names_text = 'val_img_names.txt'

In [57]:
# load img_names.txt

def get_image_names(filepath):
    file = load_doc(filepath)
    imgs = file.split(',')[:-1]

    output = []

    for f in imgs:
        path = ''
        path = f[2:len(f)-1]
        output.append(path)

    return output

#load the clean descriptions

def get_clean_descriptions(filepath, images):
    #load the cleaned up descriptions
    file = load_doc(filepath)
    descriptions = {}

    for line in file.split('\n'):
    
        words = line.split()
        if len(words) < 1:
            continue
    
        image, caption = words[0], words[1:]

        if image in images:
            if image not in descriptions:
                descriptions[image] = []

            desc = '<start> ' + " ".join(caption) + ' <end>'
            descriptions[image].append(desc)

    return descriptions

#load the features

def load_features(imgs):
    #load all the features!
    all_features = load(open('data/features.p', 'rb'))

    #then get only the needed features
    features = {k:all_features[k] for k in imgs}
    return features

# function to load text file into memory

def load_doc(filepath):
    #open a file as read only
    file = open(filepath, 'r')
    text = file.read()
    file.close()
    return text    

# convert dict to clean list of captions

def d_to_l(desc):
    corpus = []

    for k in desc.keys():
        [corpus.append(text) for text in desc[k]]

    return corpus

def create_tokenizer(desc):
    corpus = d_to_l(desc)

    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(corpus)

    return tokenizer, max(len(text.split()) for text in corpus)

In [47]:
train_imgs = get_image_names(img_names_text)
val_imgs = get_image_names(val_img_names_text)

In [48]:
train_imgs[0]

'VizWiz_train_00000000.jpg'

In [50]:
all_img_names = train_imgs + val_imgs

len(all_img_names)

31179

In [58]:
train_descriptions = get_clean_descriptions('images_with_associated_captions.txt', train_imgs)
val_descriptions = get_clean_descriptions('val_imgs_w_caps.txt', val_imgs)

In [59]:
all_descriptions = {**train_descriptions, **val_descriptions}
len(all_descriptions)

31179

In [60]:
#len(val_descriptions)

In [61]:
tokenizer = create_tokenizer(all_descriptions)

tokens = tokenizer[0]
longest_caption = tokenizer[1]

In [62]:
lex_count = len(tokens.word_index)+1

In [63]:
lex_count, longest_caption

(20468, 155)

In [64]:
# create i/o sequence pairs from the iamge description

def data_generator(captions, features, tokenizer, longest):
    while 1:
        for k, cap_list in captions.items():
            #retrieve photo features
            feature = features[k][0]
            img_in, seq, word_out = make_seq(tokenizer, longest, cap_list, feature)

            yield [[img_in, seq], word_out]

def make_seq(tokenizer, longest, cap_list, feature):
    X1, X2, y = [], [], []

    #iterate through each caption for a given image
    for cap in cap_list:
        #encode the sequence
        seq = tokenizer.texts_to_sequences([cap])[0]

        #split one sequence into multiple X,y pairs
        for i in range(1, len(seq)):
            #split into i/o pair
            in_seq, out_seq = seq[:i], seq[i]
            #pad input
            in_seq = pad_sequences([in_seq], maxlen=longest)[0]
            #encode output
            out_seq = to_categorical([out_seq], num_classes=lex_count)[0]

            #store
            X1.append(feature)
            X2.append(in_seq)
            y.append(out_seq)

    return np.array(X1), np.array(X2), np.array(y) 

In [65]:
# captioning model

def setup(lex_count, longest_caption):
  
    #compress number of features from cnn
    cnn_feats = Input(shape=(2048,))
    features_1 = Dropout(0.5)(cnn_feats)
    features_2 = Dense(256, activation='relu')(features_1)

    #lstm sequence model
    lstm_in = Input(shape=(longest_caption,))
    seq_1 = Embedding(lex_count, 256, mask_zero=True)(lstm_in)
    seq_2 = Dropout(0.5)(seq_1)
    seq_33 = LSTM(256)(seq_2)

    #merge the models
    decode_1 = add([features_2, seq_3])
    decode_2 = Dense(256, activation='relu')(decode_1)
    outputs = Dense(lex_count, activation='softmax')(decode_2)

    #model it all
    model = Model(inputs=[cnn_feats, lstm_in], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    #summarize
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)

    return model

In [66]:
all_features = {**training_features, **validation_features}

In [67]:
# TRAIN IT, FINALLY

print(f'The training dataset has {len(all_img_names)} images in it.')
print(f'There are {len(all_descriptions)} associated captions; 5 for each image.')
print(f'{len(all_features)} features were extracted.')
print(f'There are {lex_count} unique words.')
print(f'The longest caption is {longest_caption} characters long.')

model = setup(lex_count, longest_caption)
epochs = 30
steps = (len(all_descriptions))*0.2

The training dataset has 31179 images in it.
There are 31179 associated captions; 5 for each image.
31192 features were extracted.
There are 20468 unique words.
The longest caption is 155 characters long.


2022-04-18 18:28:25.386242: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-18 18:28:25.386271: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-18 18:28:25.386287: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (shpatzenbook): /proc/driver/nvidia/version does not exist
2022-04-18 18:28:25.386894: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 155)]        0           []                               
                                                                                                  
 input_1 (InputLayer)           [(None, 2048)]       0           []                               
                                                                                                  
 embedding (Embedding)          (None, 155, 256)     5239808     ['input_2[0][0]']                
                                                                                                  
 dropout (Dropout)              (None, 2048)         0           ['input_1[0][0]']                
                                                                                              

In [ ]:
#os.mkdir('models')
for i in range(20,epochs):
    generator = data_generator(all_descriptions, all_features, tokens, longest_caption)
    model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)
    model.save('models/model_' + str(i) + '.h5')

2443/6235 [==========>...................] - ETA: 31:24 - loss: 2.8094 - accuracy: 0.4381